<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%A7%E0%B8%B4%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B2%E0%B8%B0%E0%B8%AB%E0%B9%8C%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
!pip install pythainlp
!pip install -U sentence-transformers

In [68]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
from sklearn.cluster import KMeans
import pandas as pd
import ast

In [69]:
kok = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่ (1)/Facebook_pre.csv',encoding='utf-8')

In [70]:
kok

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,99แคมป์,ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เ...,0,0,13,"['ทาน', 'เนื้อ', 'ญาติพี่น้อง', 'รอบข้าง', 'มะ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,ไม่ระบุเพศ,...,0,0,0,0,0,0,0,0,1,0
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,0,0,95,"['ปี', '62', 'แม่', 'เกร็ด', 'เลือด', 'ต่ำ', '...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,A-Chiraya Kasiyapong,Bureerat Saisudta,0,0,3,"['Bureerat', 'Saisudta', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,AI'Jack TheRam,ดีมากๆเลยครับ จะได้เป็นวิทยาทานให้คนอื่น พ่อกั...,0,0,88,"['ดีมาก', 'วิทยาทาน', 'คนอื่น', 'พ่อ', 'แม่', ...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
4,AS Kate-Duan,.,0,0,1,['end'],ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,ไอ มิส ยู,อันนี้ไม่รู้จริงๆนะคะ เราไม่สามารถตรวจครั้งแรก...,0,1,51,"['อันนี้', 'รู้', 'นะคะ', 'ตรวจ', 'ครั้งแรก', ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2120,ไอด้า ไอรดาา,สมปอง มานุ,0,0,5,"['สม', 'ปอง', 'นุ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2121,ไอลดา ปรีเอี่ยม,ชาวสวน ส้ม มาอ่าน,0,0,6,"['ชาวสวน', 'ส้ม', 'อ่าน', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2122,杨小龙,คุณแม่พึ่งพบว่าเป็นมะเร็งรังไข่ตอนตุลาคมปีนี้ค...,0,1,83,"['คุณแม่', 'มะเร็ง', 'รังไข่', 'ตอน', 'ตุลาคม'...",มะเร็งรังไข่,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [71]:
len(kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'])

573

In [72]:
a=kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

In [73]:
data = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่ (1)/Facebook_pre.csv',encoding='utf-8')

In [74]:
comment=data

In [75]:
# comment=data.groupby('name').sum().reset_index()
# # สร้าง set ข้อมูลภาษาไทย
# words = set(thai_words())
# words.remove("โรคมะเร็ง")
# name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
# for i in name:
#     words.add(i)
# def remove_stopthai(tokens):
#     final = [word.lower()
#             for word in tokens if word not in thai_stopwords()]
#     return final
# def remove_bark(tokens):
#     final = [word.lower() for word in tokens if word not in [' ','(',')','@','#','-','...','.','=','+','..','1','2','3','4','5','6','7','8','9','0','']]
#     return final
# # สร้าง list เก็บตัว nlp เพิ่อนำไปวิเคราะห์โรค อาการ เเละเพศ
# list_token =[]
# count = []
# for i in range(len(comment)):#len(comment)
#     text= comment['comments'][i]
#     custom_tokenizer = Tokenizer(words)
#     Token = custom_tokenizer.word_tokenize(normalize(str(text)))
#     Token.append('end')
#     Token_final = remove_bark(remove_stopthai(Token))
#     list_token.append(Token_final)
#     count.append(len(Token_final))
# comment['จำนวนคำ'] = count
# comment['token'] = list_token

In [76]:
set_cancer=list(set(comment['defind_cancer_with_nlp']))

In [77]:
set_cancer

['มะเร็งลำไส้',
 'มะเร็งไขกระดูก',
 'มะเร็งช่องปาก',
 'มะเร็งมดลูก',
 'มะเร็งเต้านม',
 'มะเร็งหลอดอาหาร',
 'มะเร็งกระดูก',
 'มะเร็งต่อมไทรอยด์',
 'มะเร็งกล่องเสียง',
 'มะเร็งเม็ดเลือด',
 'มะเร็งตับ',
 'มะเร็งลำไส้ใหญ่',
 'มะเร็งปอด',
 'มะเร็งกระเพาะปัสสวะ',
 'มะเร็งไต',
 'มะเร็งกระเพาะอาหาร',
 'มะเร็งถุงน้ำดี',
 'มะเร็งท่อน้ำดี',
 'มะเร็งเยื่อบุโพรงมดลูก',
 'มะเร็งกระเพาะปัสสาวะ',
 'มะเร็งต่อมน้ำเหลือง',
 'มะเร็งสมอง',
 'มะเร็งผิวหนัง',
 'มะเร็งต่อมหมวกไต',
 'มะเร็งต่อมลูกหมาก',
 'มะเร็งกระดูกสันหลัง',
 'มะเร็งเม็ดเลือดขาว',
 'มะเร็งปากมดลูก',
 'เล่ามากกว่า 2 โรค',
 'มะเร็งตับอ่อน',
 'ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น',
 'มะเร็งรังไข่']

#วิธีการรักษา

In [78]:
comment

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,99แคมป์,ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เ...,0,0,13,"['ทาน', 'เนื้อ', 'ญาติพี่น้อง', 'รอบข้าง', 'มะ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,ไม่ระบุเพศ,...,0,0,0,0,0,0,0,0,1,0
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,0,0,95,"['ปี', '62', 'แม่', 'เกร็ด', 'เลือด', 'ต่ำ', '...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,A-Chiraya Kasiyapong,Bureerat Saisudta,0,0,3,"['Bureerat', 'Saisudta', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,AI'Jack TheRam,ดีมากๆเลยครับ จะได้เป็นวิทยาทานให้คนอื่น พ่อกั...,0,0,88,"['ดีมาก', 'วิทยาทาน', 'คนอื่น', 'พ่อ', 'แม่', ...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
4,AS Kate-Duan,.,0,0,1,['end'],ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119,ไอ มิส ยู,อันนี้ไม่รู้จริงๆนะคะ เราไม่สามารถตรวจครั้งแรก...,0,1,51,"['อันนี้', 'รู้', 'นะคะ', 'ตรวจ', 'ครั้งแรก', ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2120,ไอด้า ไอรดาา,สมปอง มานุ,0,0,5,"['สม', 'ปอง', 'นุ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2121,ไอลดา ปรีเอี่ยม,ชาวสวน ส้ม มาอ่าน,0,0,6,"['ชาวสวน', 'ส้ม', 'อ่าน', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
2122,杨小龙,คุณแม่พึ่งพบว่าเป็นมะเร็งรังไข่ตอนตุลาคมปีนี้ค...,0,1,83,"['คุณแม่', 'มะเร็ง', 'รังไข่', 'ตอน', 'ตุลาคม'...",มะเร็งรังไข่,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [80]:
comment['token'][1]

"['ปี', '62', 'แม่', 'เกร็ด', 'เลือด', 'ต่ำ', 'ค่า', 'ตับ', 'ผิดปกติ', 'ตรวจ', 'ไวรัส', 'ตับอักเสบ', 'ซี', 'ตับแข็ง', 'สิทธิ์', 'เบิก', 'ข้าราชการ', 'รักษา', 'รพ.', 'ศิริราช', 'ทาน', 'ยา', 'เดือน', 'ค่า', 'ไวรัส', 'อยู่ในเกณฑ์', 'ปกติ', 'F', '/U', 'U', '/S', 'เดือน', 'โควิด', 'Loss', 'F', '/U', 'อีกที', 'ปี', '64', 'U', '/S', 'HCC', 'stage', 'ทำ', 'TACE', '+...', 'ดู', 'end']"

In [82]:
fix_word = []
for i in range(len(comment)):
  x = ast.literal_eval(comment['token'][i])
  fix_word.append(x)
comment['token']=fix_word

In [84]:
comment['token'][1]

['ปี',
 '62',
 'แม่',
 'เกร็ด',
 'เลือด',
 'ต่ำ',
 'ค่า',
 'ตับ',
 'ผิดปกติ',
 'ตรวจ',
 'ไวรัส',
 'ตับอักเสบ',
 'ซี',
 'ตับแข็ง',
 'สิทธิ์',
 'เบิก',
 'ข้าราชการ',
 'รักษา',
 'รพ.',
 'ศิริราช',
 'ทาน',
 'ยา',
 'เดือน',
 'ค่า',
 'ไวรัส',
 'อยู่ในเกณฑ์',
 'ปกติ',
 'F',
 '/U',
 'U',
 '/S',
 'เดือน',
 'โควิด',
 'Loss',
 'F',
 '/U',
 'อีกที',
 'ปี',
 '64',
 'U',
 '/S',
 'HCC',
 'stage',
 'ทำ',
 'TACE',
 '+...',
 'ดู',
 'end']

In [85]:
Treatment = ['ผ่าตัด','คีโม','ฉายเเสง','แร่']
k = []
for i in range(len(comment)):
  L=[]
  for j in Treatment:
    if j in comment['token'][i]:
      L.append(j)
    else:
      L.append('ไม่สามารถระบุได้')
  L =list(set(L))
  k.append(L)
comment['Treatment'] = k

In [86]:
len(k)

2124

In [ ]:
comment[comment['Treatment'] == 'ผ่าตัด']

In [88]:
df= pd.Series(comment['Treatment'])

In [89]:
len(comment)

2124

In [90]:
Treatment_dumm = comment['Treatment'].str.join(sep='*').str.get_dummies(sep='*')
comment_data = comment.join(Treatment_dumm)

12 = มะเร็งกล่องเสียง
19 = มะเร็งไทรอยด์
31 = มะเร็งโพรงจมูก
มะเร็งไขกระดูก


#เเบ่งคำ

In [91]:
from nltk.probability import FreqDist
import plotly.graph_objects as go
import plotly.express as px
lung_cancer=comment['token']#[comment['defind_cancer_with_nlp']=='มะเร็งปอด'].reset_index()
sum_word = []
for i in range(len(lung_cancer)):
    sum_word+=lung_cancer[i]
    sum_word.remove('end')
sum_word
fdist = FreqDist(sum_word)
x=fdist.most_common(10)
word_count_table = pd.DataFrame(x,columns=['word','count'])

In [92]:
fig = px.bar(word_count_table, x='count', y='word',text_auto=True,orientation='h')
fig.show()

#จัดกลุ่ม (sentence_transformers)

In [ ]:
from sentence_transformers import SentenceTransformer,util
import numpy as np
sentences = list(comment['comments'])
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(sentences)
#Normalize the embeddings to unit length
Normalize_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
clustering_model = KMeans(n_clusters=4)
clustering_model.fit(embeddings)
cluster_assignment = clustering_model.labels_
print(cluster_assignment)
print(len(cluster_assignment))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
clusternd_sentences= {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
  if cluster_id not in clusternd_sentences:
    clusternd_sentences[cluster_id] = []

  clusternd_sentences[cluster_id].append(sentences[sentence_id])

In [ ]:
clusternd_sentences

#รวม Reddit

In [ ]:
# # prompt: ทำการ concat Data_scraper_save (6) ถึง  Data_scraper_save (17)

# import pandas as pd

# # Assuming your files are named Data_scraper_save (6).csv, Data_scraper_save (7).csv, ... , Data_scraper_save (17).csv
# # Replace with your actual file paths if needed

# df_list = []
# for i in range(6, 114):
#   file_path = f'/content/Data_scraper_save ({i}).csv'  # Replace with your actual file path
#   try:
#     df = pd.read_csv(file_path, encoding='utf-8')
#     df_list.append(df)
#   except FileNotFoundError:
#     print(f"File not found: {file_path}")

# # Concatenate all DataFrames
# final_df = pd.concat(df_list, ignore_index=True)
# final_df_drop = final_df.drop_duplicates(subset=['comments'])
# # Now you have a single DataFrame containing data from all files
# print(final_df_drop)


In [ ]:
# final_df_drop.to_csv('reddit_twothosan.csv',index=False)

In [ ]:
# final_df_drop.groupby('name').sum().reset_index()

In [ ]:
# # prompt: จากข้อมูล final_df_drop ช่วยสร้าง list เก็บชื่อที่ซ้ำ

# duplicate_names = []
# for name, group in final_df_drop.groupby('name'):
#   if len(group) > 1:
#     duplicate_names.append(name)


In [ ]:
# final_df_drop[final_df_drop['name']=='myersmjsc']

# เพิ่มเติม/note

In [ ]:
for i in range(len(comment)):
  print('ผ่าตัด' in comment['token'][i])

In [ ]:
k = pd.DataFrame()
l = []
for i in range(len(comment)):
  if 'แนะนำ' in comment['token'][i]:
    l.append(comment['token'][i])
k['word'] = l
k

In [ ]:
df1 = pd.read_csv('/content/Data_scraper_save (5).csv', encoding='utf-8')
df2 = pd.read_csv('/content/Data_Docter_ศ.นพ.ชวลิต.csv', encoding='utf-8')
df3 = pd.read_csv('/content/Data_โรคร้ายผ่านได้ถ้าไม่ยอมแพ้.csv', encoding='utf-8')

In [ ]:
# prompt: รวม df1 df2 df3

# Concatenate the DataFrames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)
combined_df.to_csv('data_Facebook.csv', index=False, encoding='utf-8')

In [ ]:
combined_df

In [ ]:
# prompt: ช่วยเเปลง '[1,2,3,4]' เป็น list [1,2,3,4]

import ast
list_string = [1,2,3,4]
try:
  list_actual = ast.literal_eval(list_string)
except:
  list_actual = list_string
print(list_actual)
